# <a name="Part4_2">2. Hyper Parameter Tuning</a>

# <a name="Part4_2_1">2.1. Hyper parameter tuning for LR - BOW Vectorizer</a>

## 1. Import and analyse the data set.

In [ ]:
import pandas as pd # read data file, data processing
import numpy as np # linear algebra
import matplotlib.pyplot as plt # plotting graph for EDA , Metrics analysis
%matplotlib inline
import seaborn as sns # plotting graph for EDA , Metrics analysis

from sklearn.pipeline import Pipeline

In [ ]:
pip install googletrans==4.0.0-rc1

     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 1.3 MB 10.5 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17416 sha256=6a2be137a7ba1d210425649b8b9d696a6b1916d76554748437a671b2e3b8bd6f
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [ ]:
import unicodedata  # Replace accented encoding characters 
from googletrans import Translator # translate given text to English text
import re # Text pre-processing


### Load the data 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import os

py_file_location = "/content/drive/MyDrive/AIML/projects/Capstone-NLP-Ticketing/"
sys.path.append(os.path.abspath(py_file_location))



In [ ]:
# User defined file for doing input preprocess for Production inputs
from InputTransformer import InputTransformer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Input data files has been processed for 
# 1. carriage return characters like '_x000D_' and \n 
# 2. Accented encoding character like äº§å“æ‰€åœ¨ä»“åº“å‡ºé”™ã€ , è¿žæŽ¥åŽè‡ªåŠ¨æ–­å¼€ï¼Œæ
# 3. Translation of words in non english language especially German, Italian, French
# Above 3 steps are done separately and output from these steps are used for further processing in Part 2
# 4. Update of Assigment group - fewer data groups , grouped to Group others
# 5. Pre-process for having only English data after translation, removal of spaces 
# 6. Treatment of Null values
# Above step 4,5,6 are done in part2 and processed data is stored in input_data_trans_preprocess.csv

data_dir = "/content/drive/MyDrive/AIML/projects/Capstone-NLP-Ticketing/"
data_file_name='input_data_trans_preprocess.csv'
#data_file_name='input_data.xlsx'
data_file_path = data_dir+data_file_name
data_file_path

'/content/drive/MyDrive/AIML/projects/Capstone-NLP-Ticketing/input_data_trans_preprocess.csv'

In [ ]:
#df_data = pd.read_excel(data_file_path)
df_data = pd.read_csv(data_file_path)

In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8467 entries, 0 to 8466
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Short description       8467 non-null   object
 1   Description             8467 non-null   object
 2   Caller                  8467 non-null   object
 3   Assignment group        8467 non-null   object
 4   orig_desc               8466 non-null   object
 5   orig_short_desc         8459 non-null   object
 6   Lang                    8467 non-null   object
 7   Translated_ShortDesc    8450 non-null   object
 8   Translated_Description  8467 non-null   object
 9   orig_assign_group       8467 non-null   object
dtypes: object(10)
memory usage: 661.6+ KB


**Feature with both description - Merging both Description and Short description**

In [ ]:
df_data.head(5)

,Short description,Description,Caller,Assignment group,orig_desc,orig_short_desc,Lang,Translated_ShortDesc,Translated_Description,orig_assign_group
0,login issue,verified user details employee manager name ch...,spxjnwir pjlcoqds,GRP_0,-verified user details.(employee# & manager na...,login issue,en,login issue,-verified user details.(employee# & manager na...,GRP_0
1,outlook,received from hmjdrvpb komuaywn gmail com hell...,hmjdrvpb komuaywn,GRP_0,_x000D_\n_x000D_\nreceived from: hmjdrvpb.komu...,outlook,en,outlook,received from: hmjdrvpb.komuaywn@gmail.com...,GRP_0
2,cant log in to vpn,received from eylqgodm ybqkwiam gmail com hi i...,eylqgodm ybqkwiam,GRP_0,_x000D_\n_x000D_\nreceived from: eylqgodm.ybqk...,cant log in to vpn,en,cant log in to vpn,received from: eylqgodm.ybqkwiam@gmail.com...,GRP_0
3,unable to access hr tool page,unable to access hr tool page,xbkucsvz gcpydteq,GRP_0,unable to access hr_tool page,unable to access hr_tool page,en,unable to access hr_tool page,unable to access hr_tool page,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error,skype error,no,skype error,skype error,GRP_0


In [ ]:
# Import stop words list from NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords # Import stop words
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
# Remove Stop words and do Lemmatization of features
def preprocess_vocab(df_column):
    corpus=[]
    stop_words=set(stopwords.words('english'))
    #stem=PorterStemmer()
    lem=WordNetLemmatizer()
    for tickets in df_column:
      words=[w for w in word_tokenize(tickets) if (w not in stop_words)]
      words=[lem.lemmatize(w) for w in words if len(w)>2]
      corpus.append(words)

    df_column_upd = df_column.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    
    return  df_column_upd, corpus

## Train a simple ML Model - Logistic Regression

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

def calc_metrics(actual,predicted):
  print('Accuracy score: ', round(accuracy_score(actual, predicted),2))
  print("precision_weighted:", round(precision_score(actual, predicted,average='weighted', zero_division=1),2))
  print("recall_weighted:", round(recall_score(actual, predicted,average='weighted', zero_division=1 ),2))
  print("f1_weighted:", round(f1_score(actual, predicted,average='weighted', zero_division=1 ),2))
  print("Classification Report:")
  print(classification_report(actual, predicted,zero_division=1))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical

def run_model(model, X_train, X_val, y_train, y_val):

  # convert X_train to BOW values  - Featurization
  vectorizer = CountVectorizer(binary=True)
  X_train_bow = vectorizer.fit_transform(X_train)
  X_val_bow = vectorizer.transform(X_val)

  # Convert Target variable to categorical value using label encoding
  y = df_data['Assignment group'].values
  le = preprocessing.LabelEncoder()
  le.fit(y)
  y_train_mdl_lbl_enc = le.transform(y_train)
  #y_train_mdl_cat = to_categorical(y_train_mdl_lbl_enc)
  y_val_mdl_lbl_enc = le.transform(y_val)
  #y_val_mdl_cat = to_categorical(y_val_mdl_lbl_enc)

  # Train the model
  #history = model.fit(X_train_bow, y_train_mdl_cat)
  history = model.fit(X_train_bow, y_train_mdl_lbl_enc)
  
  print("Model trainded")
  # Predict for given validation value

  y_pred_bow = model.predict(X_val_bow)
  print("Model Prediction completed")
  #actual = y_val_mdl_cat
  actual = y_val_mdl_lbl_enc
  predicted = y_pred_bow

  calc_metrics(actual,predicted)
  return history

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

def run_LR_model(X_train, X_val, y_train, y_val):
  clf = LogisticRegression(solver='lbfgs', max_iter=250)
  clf = OneVsRestClassifier(clf)
  #model = clf
  history = run_model(clf, X_train, X_val, y_train, y_val)
  return clf, history

In [ ]:
#Move all the Group with less than 10 rows to Group others
def targer_Preprocess(df_data):
  df_data['orig_assign_group'] = df_data['Assignment group']
  df_assign_group = pd.DataFrame(df_data['Assignment group'].value_counts())
  df_assign_group_lt10 = df_assign_group[df_assign_group['Assignment group'] < 10]
  #Move all the Group with less than 10 rows to Group others
  df_data['Assignment group'] = df_data['Assignment group'].apply( lambda x: 'GRP_Other' if x in df_assign_group_lt10.index else x)
  return df_data

#Processing already done in the previous preprocessing file step, hence skipped 
#df_data = targer_Preprocess(df_data)


In [ ]:
# missing value treatment 
def missing_value_treatment(df_data):
  cond1 = df_data['Short description'].isnull()
  cond2 = df_data['Description'].isnull()
  df_missing = df_data[ cond1 | cond2 ]
  # Replace Null description with Short description
  df_data['Description'].fillna(df_data['Short description'], inplace=True)
  # Replace Null Short description with Description
  df_data['Short description'].fillna(df_data['Description'], inplace=True)
  df_data.drop(index = df_data[ cond1 & cond2 ].index, inplace=True, axis=0)
  return df_data

#Processing already done in the previous preprocessing file step, hence skipped 
#df_data = missing_value_treatment(df_data)


In [ ]:
# Create a new column by merging both description field and use this for model training and prediction
df_data['Desc_All'] = df_data['Short description'] + ' '+ df_data['Description']

####Model : Logistic Regression ; Feature type : Bag of words;  Features :  Description & Short Description


In [ ]:
feature_name = "Desc_All"
#X= df_data[feature_name]
X, corpus = preprocess_vocab(df_data[feature_name])
y = df_data['Assignment group'].values
# Split data into Train, and Test - Test data would be used for testing the model 
X_train, X_prod, y_train, y_prod = train_test_split(X, y, test_size=0.05, random_state=0, stratify=y)
print('Prod Shape', X_prod.shape )
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, stratify=y)
print('Train shape', len(X_train))
print('Val shape', len(X_val))



Prod Shape (424,)
Train shape 6350
Val shape 2117


In [ ]:
bowVectorizer = CountVectorizer(binary=True)

In [ ]:
clf_pipeline = Pipeline( steps = [ 
                            ('vectorizer', bowVectorizer), 
                            ('clf_LR', OneVsRestClassifier(LogisticRegression(solver='lbfgs', max_iter=250)))
                            ])

In [ ]:
# Convert Target variable to categorical value using label encoding
y = df_data['Assignment group'].values
le = preprocessing.LabelEncoder()
le.fit(y)
num_classes = len(le.classes_)
y_train_mdl_lbl_enc = le.transform(y_train)
y_train_mdl_cat = to_categorical(y_train_mdl_lbl_enc, num_classes)
y_val_mdl_lbl_enc = le.transform(y_val)
y_val_mdl_cat = to_categorical(y_val_mdl_lbl_enc, num_classes)

In [ ]:
# Preprocessing already done in previous steps, henc skipped
#X_train1 = preprocess_input(X_train)

In [ ]:
clf_pipeline.fit(X_train, y_train_mdl_lbl_enc)

Pipeline(steps=[('vectorizer', CountVectorizer(binary=True)),
                ('clf_LR',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=250)))])

In [ ]:
y_pred = clf_pipeline.predict(X_val)

In [ ]:
y_val_mdl_lbl_enc = le.transform(y_val)
y_actual= y_val_mdl_lbl_enc

In [ ]:
calc_metrics(y_actual,y_pred)

Accuracy score:  0.67
precision_weighted: 0.66
recall_weighted: 0.67
f1_weighted: 0.63
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.94      0.81       994
           1       0.67      0.25      0.36         8
           2       0.67      0.40      0.50        35
           3       0.00      0.00      0.00         8
           4       0.55      0.50      0.52        64
           5       0.56      0.56      0.56        36
           6       0.79      0.37      0.50        30
           7       0.50      0.20      0.29        10
           8       0.30      0.14      0.19        21
           9       0.95      1.00      0.98        20
          10       0.61      0.50      0.55        22
          11       0.50      0.26      0.34        54
          12       0.48      0.40      0.44        60
          13       0.00      0.00      0.00         9
          14       0.50      0.14      0.22         7
          15       0.50  

In [ ]:
prod1 = "amssm00345: c:\ label:sys-amssm00345 94ef9088 on server: is over  85% space consumed. space available: 2.7 g "

In [ ]:
prod1 = X_prod.iloc[3]
print(prod1)
y_prod[3]

unable inwarehouse tool received nuhfwplj ojcwxser gmail com unable force inwarehouse tool also consigned material needs inwarehouse tool end today please help cid image png sid b


'GRP_13'

In [ ]:
X_prod1 = pd.Series(prod1)


In [ ]:
X_prod1 = InputTransformer.preprocess_input(X_prod1)

##########################Inside process input##########################
Inside process input -after remove_whitespace_CR 0    unable inwarehouse tool received nuhfwplj ojcw...
dtype: object
Inside process input -after remove_accents 0    unable inwarehouse tool received nuhfwplj ojcw...
dtype: object
Inside process input -after Translate_text 0    unable inwarehouse tool received nuhfwplj ojcw...
dtype: object
Inside process input -after Preprocessing 0    unable inwarehouse tool received nuhfwplj ojcw...
dtype: object
Inside process input -after Vocab Preprocessing 0     unable inwarehouse tool received nuhfwplj ojc...
dtype: object
##########################nside process input completed##########################


In [ ]:
y_pred = clf_pipeline.predict(X_prod1)

In [ ]:
le.inverse_transform(y_pred)

array(['GRP_13'], dtype=object)

In [ ]:
#model3, history3 = run_LR_model(X_train, X_val, y_train, y_val )

In [ ]:
# Save the pipeline as a pickle file.
import pickle

pickle.dump(clf_pipeline, open(data_dir+"saved_pipeline_lr.pkl", 'wb'))

In [ ]:
#pickle.dump(ExperimentalTransformer,  open(data_dir+"saved_ExperimentalTransformer.pkl", 'wb'))

In [ ]:
# save label encoder
pickle.dump(le, open(data_dir+"label_encoder.pkl", 'wb'))

In [ ]:
print("Lets try to use the stored Pickled pipeline\n")
# Load the pickled model
pickled_pipeline = pickle.load(open(data_dir+"saved_pipeline_lr.pkl", 'rb'))


Lets try to use the stored Pickled pipeline



In [ ]:
pickled_le = pickle.load(open(data_dir+"label_encoder.pkl", 'rb'))

In [ ]:
print(X_prod)

6053    scanning recording comes message path exist sc...
4145    expense report error manager needs authorizati...
4252    dell system slow mscrm slow dell system slow c...
2624    unable inwarehouse tool received nuhfwplj ojcw...
3426    erp pur wrong subcontracting demand nd materia...
                              ...                        
7576     network drives disconnected unable connect drive
212     please give chrtyad access reporting tool ltab...
3380    logon balancing error erp even connecting vpn ...
836     citrix access received lenxvcbq vwnhjtoi gmail...
7471    unable connect network laptop unable connect n...
Name: Desc_All, Length: 424, dtype: object


In [ ]:
#X_prod_bow = pickled_vectorizer.transform(X_prod)

In [ ]:
#print(X_prod_bow)

In [ ]:
print("\nPrediction from the pickel model for the input feature values\n")
# Use the loaded pickled model to make predictions
y_pred = pickled_pipeline.predict(X_prod)


Prediction from the pickel model for the input feature values



In [ ]:
#print(y_pred)


In [ ]:
result_lbl_enc = pickled_le.inverse_transform(y_pred)
print(result_lbl_enc[0:10])

['GRP_0' 'GRP_0' 'GRP_0' 'GRP_13' 'GRP_29' 'GRP_13' 'GRP_8' 'GRP_0'
 'GRP_3' 'GRP_0']


In [ ]:
actual = y_prod

In [ ]:
actual[0:10]

array(['GRP_33', 'GRP_2', 'GRP_0', 'GRP_13', 'GRP_29', 'GRP_13', 'GRP_8',
       'GRP_0', 'GRP_3', 'GRP_0'], dtype=object)

#### Hyper parameter tuning results - Model : LR Vectorizer : BOW

In [ ]:
clf_pipeline_hyper_tuning = Pipeline( steps = [ 
                            ('vectorizer', bowVectorizer), 
                            ('clf_LR', OneVsRestClassifier(LogisticRegression()))
                            ])




In [ ]:
clf_pipeline_hyper_tuning.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vectorizer', 'clf_LR', 'vectorizer__analyzer', 'vectorizer__binary', 'vectorizer__decode_error', 'vectorizer__dtype', 'vectorizer__encoding', 'vectorizer__input', 'vectorizer__lowercase', 'vectorizer__max_df', 'vectorizer__max_features', 'vectorizer__min_df', 'vectorizer__ngram_range', 'vectorizer__preprocessor', 'vectorizer__stop_words', 'vectorizer__strip_accents', 'vectorizer__token_pattern', 'vectorizer__tokenizer', 'vectorizer__vocabulary', 'clf_LR__estimator__C', 'clf_LR__estimator__class_weight', 'clf_LR__estimator__dual', 'clf_LR__estimator__fit_intercept', 'clf_LR__estimator__intercept_scaling', 'clf_LR__estimator__l1_ratio', 'clf_LR__estimator__max_iter', 'clf_LR__estimator__multi_class', 'clf_LR__estimator__n_jobs', 'clf_LR__estimator__penalty', 'clf_LR__estimator__random_state', 'clf_LR__estimator__solver', 'clf_LR__estimator__tol', 'clf_LR__estimator__verbose', 'clf_LR__estimator__warm_start', 'clf_LR__estimator', 'clf_LR__n_jobs']

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_param = [{
                'vectorizer__lowercase': [True],
                'vectorizer__strip_accents': ['unicode'],
                'vectorizer__ngram_range': [(1,1), (1, 2), (1, 3)],    
                'clf_LR__estimator__C': [100, 10, 1, 0.1, 0.01, 0.001, 0.0001],
                'clf_LR__estimator__penalty': ['l2'],
                'clf_LR__estimator__max_iter': [250],
               'clf_LR__estimator__solver' : ['lbfgs', 'liblinear']
               }]

gridsearch = GridSearchCV(clf_pipeline_hyper_tuning, grid_param, cv=5, scoring='f1_weighted', verbose=0,n_jobs=-1) 
best_model = gridsearch.fit(X_train,y_train_mdl_lbl_enc)


In [ ]:
#sklearn.metrics.SCORERS.keys()

In [ ]:
best_model.best_params_

{'clf_LR__estimator__C': 10,
 'clf_LR__estimator__max_iter': 250,
 'clf_LR__estimator__penalty': 'l2',
 'clf_LR__estimator__solver': 'liblinear',
 'vectorizer__lowercase': True,
 'vectorizer__ngram_range': (1, 1),
 'vectorizer__strip_accents': 'unicode'}

In [ ]:
best_model.score(X_val,y_val_mdl_lbl_enc)

0.6323517406952648

In [ ]:
predictions = gridsearch.predict(X_val)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

def calc_metrics(actual,predicted):
  print('Accuracy score: ', round(accuracy_score(actual, predicted),2))
  print("precision_weighted:", round(precision_score(actual, predicted,average='weighted', zero_division=1),2))
  print("recall_weighted:", round(recall_score(actual, predicted,average='weighted', zero_division=1 ),2))
  print("f1_weighted:", round(f1_score(actual, predicted,average='weighted', zero_division=1 ),2))
  print("Classification Report:")
  print(classification_report(actual, predicted,zero_division=1))

In [ ]:
calc_metrics(y_val_mdl_lbl_enc,predictions)

Accuracy score:  0.67
precision_weighted: 0.65
recall_weighted: 0.67
f1_weighted: 0.63
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.92      0.82       994
           1       0.29      0.25      0.27         8
           2       0.58      0.40      0.47        35
           3       0.00      0.00      0.00         8
           4       0.58      0.53      0.55        64
           5       0.54      0.53      0.54        36
           6       0.61      0.37      0.46        30
           7       0.50      0.20      0.29        10
           8       0.42      0.24      0.30        21
           9       0.95      0.95      0.95        20
          10       0.54      0.59      0.57        22
          11       0.36      0.24      0.29        54
          12       0.46      0.42      0.44        60
          13       0.25      0.11      0.15         9
          14       0.50      0.14      0.22         7
          15       0.25  